In [2]:
import torch
import cv2
import yaml
from acquisition import VideoFeedReader
import time

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s').to(device) # or yolov3-spp, yolov3-tiny, custom
#model = torch.hub.load('ultralytics/yolov5', 'yolov5x').to(device) # or yolov3-spp, yolov3-tiny, custom

Using device: cuda


Using cache found in /home/sach/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.6.9 torch-1.10.1 CUDA:0 (NVIDIA GeForce RTX 3080, 10015MiB)



WARNING ⚠️ Python 3.7.0 is required by YOLOv5, but Python 3.6.9 is currently installed


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [7]:
import numpy as np
from PIL import Image
from tqdm import tqdm

path = "data/img_5_1/img_5_1_"
nb_frames = 500

ball_class = 32
person_class = 0

use_rgb = True

codec = cv2.VideoWriter_fourcc(*'mp4v')
# Detection stream
dec_stream = cv2.VideoWriter("detection.mp4", codec, 25.0, (1280, 720), isColor=use_rgb)
# Segmentation stream
seg_stream = cv2.VideoWriter("segmentation.mp4", codec, 25.0, (1280, 720), isColor=use_rgb)
# Background stream
bg_stream = cv2.VideoWriter("background.mp4", codec, 25.0, (1280, 720), isColor=use_rgb)

read_times = []
inf_times = []
draw_times = []

display = False

for i in tqdm(range(0, nb_frames)):
    t = time.time()
    img_path = path + str(i).zfill(4) + '.jpg'
    cv_img = cv2.imread(img_path, cv2.IMREAD_COLOR if use_rgb else cv2.IMREAD_GRAYSCALE)
    read_times.append(time.time() - t)
    t = time.time()
    res = model(cv_img).xyxy[0]
    res = res[(res[:, 5] == ball_class) | (res[:, 5] == person_class)]

    inf_times.append(time.time() - t)
    #print("Inference time: ", inf_times[-1])
    t = time.time()

    # Create a mask for bounding boxes:
    mask = np.zeros((720, 1280, 3), dtype=np.uint8)
    for box in res:
        c = int(box[5])
        if c == ball_class:
            # Draw a disk
            # Center = middle of bounding box
            center = (int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2))
            # Radius = half of the bounding box
            radius = int((box[2] - box[0]) / 2)
            cv2.circle(mask, center, radius, (1, 1, 1), -1)
        else:
            # Person: draw a rectangle.
            #mask[int(box[1]):int(box[3]), int(box[0]):int(box[2]), :] = 1
            cv2.ellipse(mask, (int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)), (int((box[2] - box[0]) / 2), int((box[3] - box[1]) / 2)), 0, 0, 360, (1, 1, 1), -1)
    seg_stream.write(mask * 255)
    bg_stream.write((1 - mask) * cv_img)

    for *xyxy, conf, cls in res:
        label = '%s %.2f' % (model.names[int(cls)], conf)
        # Draw box on image
        cv2.rectangle(cv_img, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
    dec_stream.write(cv_img)

    if display:
        cv2.imshow("Detecting Motion...", cv_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    draw_times.append(time.time() - t)
cv2.destroyAllWindows()
seg_stream.release()
dec_stream.release()
bg_stream.release()

# Average inference & draw time
print("Average read time: ", np.mean(read_times))
print("Average inference time: ", np.mean(inf_times))
print("Average draw time: ", np.mean(draw_times))
# Average FPS:
print("Average FPS: ", nb_frames / (np.sum(inf_times) + np.sum(draw_times) + np.sum(read_times)))

100%|██████████| 500/500 [00:15<00:00, 31.84it/s]

Average read time:  0.009254437446594238
Average inference time:  0.007653766632080078
Average draw time:  0.014380115985870362
Average FPS:  31.960808312402197
